In [4]:
import sys
sys.path.append('../..')
import torch
import torchaudio

from torchaudio.models.decoder import ctc_decoder
from torchaudio.datasets import LIBRISPEECH

from src.datasets.unimodal import get_raw_librispeech_dataset, RawLibriLightDataset, RawLibrispeechDataset
from config import DATA_PATH
from tqdm import tqdm

In [5]:
with open('../../data/wer-scoring/lexicon-lower.txt', 'r') as file:
        lines = file.readlines()
    
with open('../../data/wer-scoring/lexicon.txt', 'w') as file:
    for line in lines:
        # Modify the line here as needed
        modified_line = line.upper()  # Example modification
        file.write(modified_line)

In [3]:
l.upper()

'ZIONISTS Z I O N I S T S |'

In [2]:
dataset = get_raw_librispeech_dataset(dataset="dev-clean", batch_size=1, shuffle=False, num_workers=1, normalize_waveform=False)

In [3]:
dataset_iter = iter(dataset)

In [4]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_10M
acoustic_model = bundle.get_model()

In [5]:
tokens = [label.lower() for label in bundle.get_labels()]
print(tokens)

['-', '|', 'e', 't', 'a', 'o', 'n', 'i', 'h', 's', 'r', 'd', 'l', 'u', 'm', 'w', 'c', 'f', 'g', 'y', 'p', 'b', 'v', 'k', "'", 'x', 'j', 'q', 'z']


In [6]:
from torchaudio.models.decoder import download_pretrained_files

files = download_pretrained_files("librispeech-4-gram")

print(files)

2024-03-05 15:33:37,518 - torchaudio.utils.download - INFO - The local file (/Users/timcares/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lexicon.txt) exists. Skipping the download.
2024-03-05 15:33:37,519 - torchaudio.utils.download - INFO - The local file (/Users/timcares/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/tokens.txt) exists. Skipping the download.
2024-03-05 15:33:37,519 - torchaudio.utils.download - INFO - The local file (/Users/timcares/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lm.bin) exists. Skipping the download.


PretrainedFiles(lexicon='/Users/timcares/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lexicon.txt', tokens='/Users/timcares/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/tokens.txt', lm='/Users/timcares/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lm.bin')


In [7]:
# lm weight and word score after https://arxiv.org/pdf/2006.11477.pdf#page10 Appendix B
# beam size after https://arxiv.org/pdf/2006.11477.pdf#page10 Section 4.4
LM_WEIGHT = 2.90
WORD_SCORE = -1.62
BEAM_SIZE = 1500

beam_search_decoder = ctc_decoder(
    lexicon=files.lexicon,
    tokens=files.tokens,
    lm=files.lm,
    beam_size=BEAM_SIZE,
    lm_weight=LM_WEIGHT,
    word_score=WORD_SCORE,
)

In [8]:
data_raw = LIBRISPEECH(root=DATA_PATH, url='dev-clean', download=True)
data_raw_iter = iter(data_raw)

In [9]:
avg_wear = 0
i=0
for batch, padding_mask in tqdm(dataset_iter, total=len(dataset_iter)):
    lengths = (1-padding_mask).sum(axis=-1)
    emissions, output_lengths = acoustic_model(batch, lengths=lengths)
    beam_search_result = beam_search_decoder(emissions, lengths=output_lengths)
    for beam_result in beam_search_result:
        actual_transcript = next(data_raw_iter)[2].lower().split()
        wer = torchaudio.functional.edit_distance(actual_transcript, beam_result[0].words) / len(actual_transcript)
        avg_wear += wer
        i+=1
avg_wear = avg_wear / i

  0%|          | 10/2703 [00:16<1:14:38,  1.66s/it]

In [ ]:
avg_wear

0.11764705882352941

In [ ]:
# # batch wise

# avg_wear = 0
# for batch, padding_mask in tqdm(dataset_iter, total=len(dataset_iter)):
#     lengths = (1-padding_mask).sum(axis=-1)
#     emissions, output_lengths = acoustic_model(batch, lengths=lengths)
#     beam_search_result = beam_search_decoder(emissions, lengths=output_lengths)
#     for beam_result in beam_search_result:
#         actual_transcript = next(data_raw_iter)[2].lower().split()
#         wer = torchaudio.functional.edit_distance(actual_transcript, beam_result[0].words) / len(actual_transcript)
#         print(beam_result[0].words)
#         print(actual_transcript)
#         avg_wear += wer
#     break
# avg_wear = avg_wear / (len(dataset_iter)*dataset.batch_size)